In [1]:
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))

import ModelFramework as mf
import FwtModels.RectWing as rw

## Create Method to Calculate the Eigen Response of a Given model

Model defined by number of b and t shapes and model ID

Using settings for JEC Wing

In [5]:
def Model_Eigen(model_num,b_modes,t_modes):
    print(model_num)
    p = rw.base_params(b_modes + t_modes + 1)
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    
    dataset_name = 'JEC'
    # Set JEC Specific parameters
    p.rho.value = 1.225
    p.s_t.value = 12
    p.c.value = 1.8
    p.EI.value = 9.77e6
    p.GJ.value = 0.99e6
    p.rho_t.value = 19.53
    p.e_0.value = 0.08
    p.e_1.value = 0
    
    p.ratio_fwt.value = 0.2
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(0,150,151))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.ratio_DL,[0,0.05,0.1,0.2]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))

    variables = [k for k,v in vars_ls]
    
    calc_fixed = True if np.isin(2,np.array([1,2,3,4])) else False
    print(model_num)
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed)   
    print(model_num)
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{model_num}_{dataset_name}.pkl')

In [6]:
b_modes = 3
t_modes = 3
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())

for k in range(8):
    pool.apply_async(Model_Eigen,args=(k,b_modes,t_modes))
pool.close()
pool.join()

0
1
2
5
4
7
3
6
6
7
0
2
1
3
4
5
1
0
2
6
3
7
4
5
